# Esercizio 6

Dato un file in formato `GTF` (Gene Transfer Format) che annota un set di geni sulla stessa genomica di riferimento, e il file della genomica di riferimento (*genomic reference*) in formato `FASTA`, produrre in output:

- l'elenco degli esoni annotati, ciascuno con il set dei trascritti di appartenenza (e il relativo gene). Gli esoni devono essere elencati per numero crescente di trascritti di appartenenza
- l'elenco degli esoni completamente coperti da coding sequence, specificandone la suddivisione in codoni in relazione ad ognuno dei trascritti di appartenenza

***

Input:

- nome del file in formato `GTF`
- nome del file della sequenza genomica in formato `FASTA`

***

Requisiti:

- deve essere definita una funzione `reverse_complement_in_case()` che prenda come argomento una sequenza nucleotidica e un valore di *strand* e ne restituisca il reverse&complement se lo strand è `-`, altrimenti restituisce la sequenza così com'è
- deve essere definita una funzione `codon_splitting()` che prenda come argomento una sequenza nucleotidica e un valore di *frame* in `{0, 1, 2}`, operi la suddivisione in codoni della sequenza (tenendo conto del valore di *frame*) e restituisca una stringa che unisca i codoni usando il carattere di spazio come separatore
- deve essere definita una funzione `get_transcript_and_gene()` che prenda come argomento una record `GTF` e restituisca una tupla contenente l'ID del trascritto come primo elemento e l'ID del gene come secondo elemento.


**NOTA BENE**: gli attributi (coppie *nome-valore*) del nono campo del file GTF non devono essere pensati a ordine fisso all'interno del campo. Per estrarre quindi un attributo, non si può usare il metodo `split()`, ma si deve necessariamente usare un'espressione regolare.

***

Variabili di output:
- `annotated_exon_list`: lista degli esoni annotati (ordinati per numero decrescente di trascritti di appartenenza); ogni elemento è una lista annidata di due elementi: tupla (*start*, *end*) dell'esone e `set` dei trascritti a cui l'esone appartiene (ogni trascritto deve essere rappresentato tramite la tupla *(transcript_id, gene_id)*)
- `exon_coverage_list`: lista degli esoni coperti interamente da coding sequence con la relativa suddivisione in codoni; ogni elemento della lista è una tupla dei seguenti cinque elementi: *transcript_id*, *gene_id*, start dell'esone, end dell'esone, suddivisione in codoni.

**NOTA BENE**: uno esone può comparire in elementi diversi della lista `exon_coverage_list` dal momento che può essere incluso in trascritti diversi.

***

### Note sul formato `GTF`

#### *Feature* e *record* `GTF`

Una *feature* `GTF` è un intervallo di posizioni sulla genomica di riferimento che ha un certo significato funzionale, ad esempio un esone (*feature* di tipo `exon`) o un frammento della coding sequence di un trascritto (*feature* di tipo `CDS`). 

Un *record* `GTF` rappresenta una *feature* di un certo tipo inclusa in un determinato trascritto (di un determinato gene). Il tipo di *feature* è specificato nel terzo campo del *record*.

Ad esempio il *record*:

    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-014"; gene_id "ARHGAP4";
    
rappresenta un esone (*feature* di tipo `exon`) che inizia in posizione `64566` e finisce in posizione `64757`, incluso nel trascritto `U52112.4-014` del gene `ARHGAP4`.

Invece il *record*:

    ENm006 VEGA_Known CDS	70312	70440	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

rappresenta un frammento della coding sequence (*feature* di tipo `CDS`) del trascritto `U52112.4-005` del gene `ARHGAP4`, mappato sulla genomica di riferimento dalla posizione `70312` alla posizione `70440`.

#### Esone incluso in trascritti diversi

Tutti i *record* di tipo `exon`, che corrispondono alla stessa *feature* sulla genomica di riferimento (cioé allo stesso intervallo di posizioni) rappresentano lo stesso esone incluso in trascritti diversi.

Ad esempio i sei *record* seguenti:

    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-014"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-002"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-003"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-001"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-024"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-011"; gene_id "ARHGAP4";
    
rappresentano l'esone `[64566, 64757]` incluso in sei trascritti diversi del gene `ARHGAP4`.

#### Esone coperto da coding sequence per un dato trascritto

Un esone di un certo trascritto è coperto completamente da coding sequence se, oltre al *record* di tipo `exon` che rappresenta l'esone incluso nel trascritto, esiste anche un *record* di tipo `CDS` corrispondente allo stesso intervallo di posizioni.

Ad esempio i due *record*:

    ENm006 VEGA_Known exon	70312	70440	.	-	.	transcript_id "U52112.4-005"; gene_id "ARHGAP4";
    ENm006 VEGA_Known CDS	70312	70440	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

indicano che l'esone `[70312, 70440]` (incluso nel trascritto `U52112.4-005`del gene `ARHGAP4`) è coperto completamente da coding sequence.

#### Suddivisione in codoni di una *feature* di tipo `CDS`

La suddivisione in codoni di una *feature* di tipo `CDS` deve tenere conto del valore del campo *frame* (ottavo campo del *record*), che specifica la posizione della prima base della *feature* all'interno del codone di appartenenza.

Ad esempio il *record* di tipo `CDS` seguente:

    ENm006 VEGA_Known CDS	70312	70440	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";
    
rappresenta un frammento della coding sequence del trascritto `U52112.4-005` del gene `ARHGAP4`. Il valore 0 del campo *frame* indica che la prima base della sequenza della *feature* è la prima base di un codone (cioé le prime tre basi della *feature* sono un codone completo).
Tenendo presente che la sequenza della *feature* estratta dalla *genomic reference* è:                      
 
    cggcaggccaagttcatggagcacaaactcaagtgcacaaaggcgcgcaacgagtacctgcttagcctggctagtgtcaacgctgctgtcagtaactactacctgcatgacgtcttggacctcatggac

la sua suddivisione in codoni sarà dunque:

    cgg cag gcc aag ttc atg gag cac aaa ctc aag tgc aca aag gcg cgc aac gag tac ctg ctt agc ctg gct agt gtc aac gct gct gtc agt aac tac tac ctg cat gac gtc ttg gac ctc atg gac

Il *record* di tipo `CDS` seguente:
    
    ENm006 VEGA_Known CDS	72521	72683	.	-	1	transcript_id "U52112.4-019"; gene_id "ARHGAP4";

rappresenta un frammento della coding sequence del trascritto `U52112.4-019` del gene `ARHGAP4`. Il valore 1 del campo *frame* indica che la prima base della sequenza della *feature* è la seconda base di un codone (cioé le prime due basi della *feature* sono le ultime due basi di un codone la cui prima base sarà l'ultima della *feature* `CDS` precedente).
Tenendo presente che la sequenza della *feature* estratta dalla *genomic reference* è:                      

    gaaggagccgtccctcctgtcgcccttgcactgctgggcggtgctgctgcagcacacgcggcagcagagccgggagagcgcggccctgagtgaggtgctggccgggcccctggcccagcgcctgagtcacattgcagaggacgtggggcgcctggtcaagaag

la sua suddivisione in codoni sarà dunque:

    ga agg agc cgt ccc tcc tgt cgc cct tgc act gct ggg cgg tgc tgc tgc agc aca cgc ggc agc aga gcc ggg aga gcg cgg ccc tga gtg agg tgc tgg ccg ggc ccc tgg ccc agc gcc tga gtc aca ttg cag agg acg tgg ggc gcc tgg tca aga ag

Il *record* di tipo `CDS` seguente:

    ENm006 VEGA_Known CDS	72761	72965	.	-	2	transcript_id "U52112.4-003"; gene_id "ARHGAP4";

rappresenta un frammento della coding sequence del trascritto `U52112.4-003` del gene `ARHGAP4`. Il valore 2 del campo *frame* indica che la prima base della sequenza della *feature* è la terza base di un codone (cioé la prima base della *feature* è l'ultima base di un codone le cui prime due basi saranno le ultime della *feature* `CDS` precedente).
Tenendo presente che la sequenza della *feature* estratta dalla *genomic reference* è:                      

    agatgcgctggcagctgagcgagcagctgcgctgcctggagctgcagggcgagctgcggcgggagttgctgcaggagctggcagagttcatgcggcgccgcgctgaggtggagctggaatactcccggggcctggaaaagctggccgagcgcttctccagccgtggaggccgcctggggagcagccgggagcaccaaagcttccg

la sua suddivisione in codoni sarà dunque:

    a gat gcg ctg gca gct gag cga gca gct gcg ctg cct gga gct gca ggg cga gct gcg gcg gga gtt gct gca gga gct ggc aga gtt cat gcg gcg ccg cgc tga ggt gga gct gga ata ctc ccg ggg cct gga aaa gct ggc cga gcg ctt ctc cag ccg tgg agg ccg cct ggg gag cag ccg gga gca cca aag ctt ccg